In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import requests
import random
import sys
import pathlib
import csv



import argparse

# import multiprocessing
from fuzzywuzzy import fuzz

# url = 'http://ai-capo-api-lb/spaCy_ner_predictor?text_sentence='  # Load Balancer

# result_path = '/nfs/gns/literature/machine-learning/evaluation/FP_Analysis/SpaCy/model_call/'
# pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)



In [2]:
!which conda

/nfs/gns/literature/Santosh_Tirunagari/miniconda3/condabin/conda


In [3]:
sys.path.insert(1, '/nfs/gns/literature/machine-learning/Santosh/Gitlab/biobertepmc/')

In [4]:
import logging
import torch
from torch.utils.data import DataLoader
import pickle
from biobert.model.bert_crf_model import BertCRF
from biobert.data_loader.epmc_loader import NERDatasetBatch
from biobert.utils.utils import my_collate
from nltk.tokenize import WordPunctTokenizer
from collections import namedtuple

In [5]:
# import spacy
# from spacy import util
# best_model_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pretrain_exp/best/'

# print("Loading from", best_model_path)
# nlp2 = util.load_model_from_path(best_model_path)

In [7]:
# data_file_path = '/nfs/misc/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/fulltext/job_14/annotation/patch-total-417.xml.gz'
data_file_path = '/nfs/misc/literature/rdf_annotation_data/daily_pipeline_api/15_08_1947/abstract/job_21/annotation/patch-total-415.abstract.gz'

In [8]:
def getfileblocks(file_path):
    subFileBlocks = []

    with gzip.open(file_path, 'rt') as fh:
        for line in fh:
            if line.startswith('<!DOCTYPE "JATS-archivearticle1.dtd">'):  # <!DOCTYPE article
                subFileBlocks.append(line)
            else:
                subFileBlocks[-1] += line

    return subFileBlocks





In [9]:
files_list = getfileblocks(data_file_path)

In [11]:
soup = BeautifulSoup(files_list[101], 'lxml')

In [ ]:
# url = 'http://ai-capo-api-3:5001/predict/sent' 

In [10]:
batch_size = 16
class MLModel:
    def __init__(self):
        self.bertCrf_model = load_model()

        # bertCrf_model.load_state_dict(torch.load('/homes/yangx/home/gitrepo/biobertepmc/model/bert_crf_model.states', map_location=device))
        self.bertCrf_model.load_state_dict(torch.load(MODEL_PATH + 'bert_crf_model.states', map_location=device))
        self.bertCrf_model.bert_model.bert_model.to(device)

    def post(self, sentences):
        BATCH_SIZE = 16
        text = sentences
        # print(text)
        with torch.no_grad():
            processor, tokens, spans = load_data_processor(text)
            dataLoader = DataLoader(dataset=processor, batch_size=BATCH_SIZE, collate_fn=my_collate, num_workers=2)

            idx2label = params['idx2label']
            self.bertCrf_model.eval()
            entities = []
            for i_batch, sample_batched in enumerate(dataLoader):
                inputs = sample_batched['input']

                bert_inputs, bert_attention_mask, bert_token_mask, wordpiece_alignment, split_alignments, lengths, token_mask \
                    = processor.tokens_totensor(inputs)

                _, preds = self.bertCrf_model.predict(input_ids=bert_inputs.to(device),
                                                      bert_attention_mask=bert_attention_mask.to(device),
                                                      bert_token_mask=bert_token_mask,
                                                      alignment=wordpiece_alignment,
                                                      splits=(split_alignments, lengths),
                                                      token_mask=token_mask)
                if idx2label:
                    for i, (path, score) in enumerate(preds):
                        labels = [idx2label[p] for p in path]
                        offset_index = i_batch * BATCH_SIZE + i
                        entities.append([[e.span[0], e.span[1], e.tag, e.text]
                                         for e in extract_entity(labels, spans[offset_index], text[offset_index])])
        return {'annotations': entities}


def load_data_processor(inputs):
    token_spans = []
    tokens = []
    for line in inputs:
        token_spans.append(list(tokenizer.span_tokenize(line)))
        tokens.append([line[start: end] for start, end in token_spans[-1]])

    processor = NERDatasetBatch.from_params(params=params, inputs=tokens)
    return processor, tokens, token_spans


def load_model():
    allowed_transitions = None
    model = BertCRF(num_tags=params['num_tags'],
                    model_name=params['model_name'],
                    stride=params['stride'],
                    include_start_end_transitions=True,
                    constraints=allowed_transitions)
    return model


def extract_entity(preds, spans, text, length=20):
    """
    extract entity from label sequence
    :param preds: a list of labels in a sentence
    :type preds: List[str
    :param spans:
    :type spans:
    :return: A list of entity object
    :rtype: List[Entity]
    """
    entities = []
    tmp = []

    for i, token in enumerate(preds):
        if token == 'O':
            pos, tag = 'O', 'O'
            label = None
        else:
            pos, tag = token.split('-')
            label = Entity_Label(index=i, pos=pos, tag=tag, span=spans[i])

        if pos in {'B', 'O'} and tmp:
            start_span = tmp[0].span[0]
            end_span = tmp[-1].span[1]
            entities.append(Entity(span=(start_span, end_span),
                                   tag=tmp[0].tag,
                                   text=text[start_span:end_span],
                                   pre=text[max(0, start_span - length):start_span],
                                   post=text[end_span: end_span + length]))
            tmp[:] = []
        if pos == 'B' or pos == 'I':
            tmp.append(label)

    if tmp:
        start_span = tmp[0].span[0]
        end_span = tmp[-1].span[-1]
        entities.append(
            Entity(span=(start_span, end_span),
                   tag=tmp[0].tag,
                   text=text[start_span:end_span],
                   pre=text[max(0, start_span - length):start_span],
                   post=text[end_span:end_span + length])
        )
    return entities


# Create a function called "chunks" with two arguments, l and n:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i + n]


def clean_Nones(ner_tags_):
    ner_tags = []
    # had to do this as the position of entity tag and entity are exchanged in CD
    for each_ner_tag in ner_tags_:
        if 'CD' == each_ner_tag[2]:
            ner_tags.append([each_ner_tag[0], each_ner_tag[1], each_ner_tag[3], each_ner_tag[2]])
        else:
            ner_tags.append(each_ner_tag)

    ner_tags = sorted(ner_tags, key=lambda x: len(x[3]), reverse=True)
    if len(ner_tags) == 1 and 'None' in ner_tags:
        return ner_tags
    elif len(ner_tags) > 1 and 'None' in ner_tags:
        ner_tags.remove('None')
        return ner_tags
    else:
        return ner_tags


In [12]:
all_sentences = soup.find_all('sent')# all_sentences = soup.find_all('SENT')

In [13]:
# soup

In [14]:
plain_sentences = []
section_tags = defaultdict(set)
gene_sentences = defaultdict(set)
disease_setences = defaultdict(set)
for each_sentence in all_sentences:
    extracted_sentence = each_sentence.plain
    if extracted_sentence:
        try:
#             section_tags[extracted_sentence.text].add(extracted_sentence.findParents('SecTag')[0]['type'])
            section_tags[extracted_sentence.text].add(extracted_sentence.findParents('sectag')[0]['type'])
        except IndexError:
            section_tags[extracted_sentence.text].add('')
        plain_sentences.append(extracted_sentence.text.strip())
        #### Extract GP sentences and their tags######
        extracted_sentence_GP_ztags = extracted_sentence.find_all('z:uniprot')
        if extracted_sentence_GP_ztags is not None:
            for each_GP_z_tag in extracted_sentence_GP_ztags:
                gene_sentences[each_sentence.plain.text].add(each_GP_z_tag.text)
        #### Extract Disease sentences and their tags######
        extracted_sentence_DS_ztags = extracted_sentence.find_all('z:disease')
        if extracted_sentence_DS_ztags is not None:
            for each_DS_z_tag in extracted_sentence_DS_ztags:
                disease_setences[each_sentence.plain.text].add(each_DS_z_tag.text)    

In [15]:
section_tags

defaultdict(set,
            {'Differential contribution of the two waves of cardiac progenitors and their derivatives to aorta and pulmonary artery. ': {''},
             'During mouse development, part of the cells derived from the second heart field (SHF) progenitors contributes to the elongation and enlargement of the outflow tract (OFT) that subsequently septates into the trunks of aorta (Ao) and pulmonary artery (PA). ': {'ABS'},
             'Thus, the cardiac progenitor-originated cells are distributed to both Ao and PA. ': {'ABS'},
             'Here, we investigated that how these cells are assigned to the two great arteries during OFT septation through lineage tracing technology. ': {'ABS'},
             'By use of the inducible Mef2c-AHF-CreERT2; Rosa26-mTmG reporter system, two waves of SHF progenitors and their derivatives were identified, and they made differential contribution to the Ao and PA, respectively. ': {'ABS'},
             'While the early wave of cells (at E7

In [60]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def get_all_tags(all_sentences):
        ML_annotations = ml_model.post(all_sentences)
        return ML_annotations['annotations']#list(filter(None, ML_annotations['annotations']))

In [19]:
MODEL_PATH = '/nfs/gns/literature/machine-learning/Santosh/Gitlab/biobertepmc/reproduce_GP_DS_OG_CD/1604049631/'

# path to the file that has model parameters
params_path = MODEL_PATH + "params.pickle"
with open(params_path, 'rb') as f:
    params = pickle.load(f)
params['max_ner_token_len'] = -1
params['max_bert_token_len'] = -1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ml_model = MLModel()

In [26]:
from collections import namedtuple

Entity = namedtuple('Entity', ['span', 'tag', 'text', 'pre', 'post'])
Entity_Label = namedtuple('Label', ['index', 'pos', 'tag', 'span'])

In [66]:
annotations = get_all_tags(plain_sentences)

In [67]:
annotations

[[],
 [[7, 12, 'OG', 'mouse']],
 [],
 [],
 [[24, 29, 'GP', 'Mef2c'],
  [30, 33, 'GP', 'AHF'],
  [34, 41, 'GP', 'CreERT2'],
  [85, 88, 'GP', 'SHF']],
 [],
 [[25, 29, 'GP', 'PDK1'], [97, 101, 'GP', 'Pdk1']],
 [[213, 231, 'DS', 'pulmonary stenosis']]]

In [29]:
import en_relationv01

In [30]:
relations = en_relationv01.load()

In [38]:
doc = relations(plain_sentences[5])

In [40]:
def roundoff(dict_y):
    for k, v in dict_y.items():
        v = round(v,2) 
        dict_y[k] = v 
    return dict_y


In [41]:
roundoff(doc.cats)

{'Yes': 0.97,
 'No': 0.0,
 'Positive': 0.16,
 'Negative': 0.07,
 'Neutral': 0.51,
 'Altered Expression': 0.03,
 'Genetic Variation': 0.0,
 'Any': 0.34,
 'Regulatory modification': 0.02}

In [79]:
ml_tagged_sentences = defaultdict(list)
count = 0
for each_sentence in tqdm(plain_sentences):
#     rs = requests.post(url, data={'text': each_sentence})
#     ml_annotations = rs.json()['annotations']
    if annotations[count]:
        ml_tagged_sentences[each_sentence].append(annotations[count])
    count = count+1
    

100%|██████████| 8/8 [00:00<00:00, 25575.02it/s]


In [80]:
ml_tagged_sentences

defaultdict(list,
            {'During mouse development, part of the cells derived from the second heart field (SHF) progenitors contributes to the elongation and enlargement of the outflow tract (OFT) that subsequently septates into the trunks of aorta (Ao) and pulmonary artery (PA).': [[[7,
                12,
                'OG',
                'mouse']]],
             'By use of the inducible Mef2c-AHF-CreERT2; Rosa26-mTmG reporter system, two waves of SHF progenitors and their derivatives were identified, and they made differential contribution to the Ao and PA, respectively.': [[[24,
                29,
                'GP',
                'Mef2c'],
               [30, 33, 'GP', 'AHF'],
               [34, 41, 'GP', 'CreERT2'],
               [85, 88, 'GP', 'SHF']]],
             'In addition, we unveiled PDK1 as a critical regulator of the second wave of cells as deletion of Pdk1 resulted in poorly developed PA leading to pulmonary stenosis.': [[[25,
                29,
    

In [ ]:
# ML not recognising covid terms in some cases.

In [81]:
missing_list = ['covid-19', 'covid19', 'sarscov2', 'sars-cov-2', '2019-ncov']
def compare_ml_annotations_with_dictionary_tagged(ml_tags_, z_tags_):
    agreed_z_tags = set()
#     print(z_tags_, ml_tags_)
    for each_z_tag in z_tags_:
        for each_ml_annotation in ml_tags_:
            if each_z_tag.lower() in missing_list:
                agreed_z_tags.add(each_z_tag)
            else:
                score = fuzz.partial_ratio(each_ml_annotation, each_z_tag) #token_set_ratio
                if score == 100:
                    agreed_z_tags.add(each_z_tag)
    return list(z_tags_- agreed_z_tags)

In [82]:
gene_sentences

defaultdict(set,
            {'By use of the inducible Mef2c-AHF-CreERT2; Rosa26-mTmG reporter system, two waves of SHF progenitors and their derivatives were identified, and they made differential contribution to the Ao and PA, respectively. ': {'AHF',
              'Mef2c'},
             'In addition, we unveiled PDK1 as a critical regulator of the second wave of cells as deletion of Pdk1 resulted in poorly developed PA leading to pulmonary stenosis. ': {'PDK1',
              'Pdk1'}})

In [83]:
all_ml_gp_tags = set()
all_ml_ds_tags = set()
all_gp_ztags = set()
all_ds_ztags = set()

for each_sentence, gp_z_tags in gene_sentences.items():
    ml_tags = ml_tagged_sentences[each_sentence]
    for each_ml_tag in ml_tags[0]:
        if each_ml_tag[2] =='GP':
            all_ml_gp_tags.add(each_ml_tag[3])
    for each_gp_ztag in gp_z_tags:
        all_gp_ztags.add(each_gp_ztag)
    
for each_sentence, ds_z_tags in disease_setences.items():
    ml_tags = ml_tagged_sentences[each_sentence]
    for each_ml_tag in ml_tags[0]:
        if each_ml_tag[2] =='DS':
            all_ml_ds_tags.add(each_ml_tag[3])
    for each_ds_ztag in ds_z_tags:
        all_ds_ztags.add(each_ds_ztag)   

IndexError: list index out of range

In [73]:
ml_tagged_sentences[each_sentence]

[]

In [ ]:
GP_false_postives = compare_ml_annotations_with_dictionary_tagged(all_ml_gp_tags, all_gp_ztags)
GP_false_postives

In [ ]:
DS_false_postives = compare_ml_annotations_with_dictionary_tagged(all_ml_ds_tags, all_ds_ztags)
DS_false_postives

In [ ]:
## End of Europe PMC with the ML FP filtering!! Save the XMLS for EuropePMC

In [74]:
import numpy as np
import itertools
import re

In [ ]:
ml_tagged_sentences['Projected healthcare resource needs for an effective response to COVID-19 in low-income and middle-income countries are significant, but an early response to limit the spread of the SARS-COV-2 virus will reduce resources needed and costs.'] = [[]]

In [ ]:
# ml_tagged_sentences

In [84]:
def get_new_missing_tags(each_sentence):
    new_entities = []
    for missing_string in missing_list:
        for i in re.finditer(missing_string, each_sentence.lower()):
            indexlocation= i.span()
    #         print(indexlocation)
            startindex= i.start()
            endindex= i.end()
            entity = each_sentence[indexlocation[0]:indexlocation[1]]
            new_entities.append([startindex,endindex, 'DS', entity])
    return new_entities

In [85]:
ml_tagged_sentences

defaultdict(list,
            {'During mouse development, part of the cells derived from the second heart field (SHF) progenitors contributes to the elongation and enlargement of the outflow tract (OFT) that subsequently septates into the trunks of aorta (Ao) and pulmonary artery (PA).': [[[7,
                12,
                'OG',
                'mouse']]],
             'By use of the inducible Mef2c-AHF-CreERT2; Rosa26-mTmG reporter system, two waves of SHF progenitors and their derivatives were identified, and they made differential contribution to the Ao and PA, respectively.': [[[24,
                29,
                'GP',
                'Mef2c'],
               [30, 33, 'GP', 'AHF'],
               [34, 41, 'GP', 'CreERT2'],
               [85, 88, 'GP', 'SHF']]],
             'In addition, we unveiled PDK1 as a critical regulator of the second wave of cells as deletion of Pdk1 resulted in poorly developed PA leading to pulmonary stenosis.': [[[25,
                29,
    

In [87]:
# get gp, ds, cd, and og sets
gp_set = set()
ds_set = set()
cd_set = set()
og_set = set()

final_sentences = defaultdict(list)

for sentence, ml_tags in ml_tagged_sentences.items():
    all_tags =[]
    new_missing_stuff = get_new_missing_tags(sentence)
    if ml_tags[0]:
        all_tags = new_missing_stuff+ml_tags[0]
    else:
        all_tags = new_missing_stuff
    if all_tags: # only if the tags are present, so discard the sentence without any tags
        final_sentences[sentence] = all_tags
        for each_ml_tag in all_tags:
            if each_ml_tag[2] =='GP':
                gp_set.add(each_ml_tag[3])
            elif each_ml_tag[2] =='DS':
                ds_set.add(each_ml_tag[3])
            if each_ml_tag[2] =='CD':
                cd_set.add(each_ml_tag[3])
            if each_ml_tag[2] =='OG':
                og_set.add(each_ml_tag[3])
        

IndexError: list index out of range

In [ ]:

def get_sentences_offset_per_tag(tagset, sentences_tags, tag_type):
    dicttag = defaultdict(list)
    for each_tag in tagset:
        for each_sentence, ml_tags in sentences_tags.items():
            for each_ml_tag in ml_tags:
                if each_tag in each_ml_tag[3] and each_ml_tag[2] ==tag_type:
                    dicttag[each_tag].append([each_sentence, each_ml_tag[0], each_ml_tag[1]])
    return dicttag


def get_mapped_list_from_annotations(annotation_list):
    mapped_list = list(itertools.combinations(annotation_list, 2))

    unique_maplist = []
    for each_list in mapped_list:
        if each_list[0][2] !=each_list[1][2] and each_list[1][2]!='OG' and each_list[0][2]!='OG':
            unique_maplist.append((each_list[0], each_list[1]))

    return unique_maplist  

def get_sentences_offset_per_cooccurance(sentences_tags):
    
    dict_gp_ds = defaultdict(list)
    dict_gp_cd = defaultdict(list)
    dict_cd_ds = defaultdict(list)

    for sentence, tags in final_sentences.items():
        if len(tags)>1: # only if more than 1 tag is available
            check_tags =np.array(tags)
            if 'GP' in  check_tags and 'DS' in check_tags:
                dict_gp_ds[sentence] = get_mapped_list_from_annotations(tags)
            if 'GP' in  check_tags and 'CD' in check_tags:
                dict_gp_cd[sentence] = get_mapped_list_from_annotations(tags)
            if 'CD' in  check_tags and 'DS' in check_tags:
                dict_cd_ds[sentence]= get_mapped_list_from_annotations(tags)         
                
    return dict_gp_ds, dict_gp_cd, dict_cd_ds



def get_article_offset_per_tag(soup, tagset):
    dicttag = defaultdict(list)
    
    
    for each_tag in tagset:
        for i in re.finditer(each_tag, each_sentence.lower()):
            indexlocation= i.span()
    #         print(indexlocation)
            startindex= i.start()
            endindex= i.end()
            entity = each_sentence[indexlocation[0]:indexlocation[1]]
        dicttag[each_tag].append([each_sentence, each_ml_tag[0], each_ml_tag[1]])
    return dicttag

In [ ]:
# 
# GP": [
# {
#   label: NR2B,
#   abstract_matches: [{start: XXXX, end: XXXX}, {start: XXXX, end: XXXX}],
#   full_text_matches: [{start: XXXX, end: XXXX}]
# },
# {
#  label: "NR2B",
#  abstract_matches:[{start:XXX, end:XXXXX}],
# },

## hard to get above notations are we process at sentences level, so the offset changes per sentence.

dict_gp = get_sentences_offset_per_tag(gp_set, final_sentences, tag_type = 'GP')
dict_ds = get_sentences_offset_per_tag(ds_set, final_sentences, tag_type = 'DS')
dict_cd = get_sentences_offset_per_tag(cd_set, final_sentences, tag_type = 'CD')


In [ ]:
dict_gp

In [ ]:
xx_ = ' '.join(plain_sentences[1:])

In [ ]:
# {
#   "lit_id": "http://europepmc.org/abstract/MED/11250803",
#   "mined_sentences": [
# 	{
#   	"section": "abstract",
#   	"t_start": 222,
#   	"t_end": 242,
#   	"d_start": 23,
#   	"d_end": 41,
#   	"text": "Both further stimulate cardiac hypertrophy and, importantly, activate counterregulatory mechanisms including overexpression of atrial natriuretic peptide and b-type natriuretic peptide, and production of cytokines such as tumor necrosis factor-alpha."
# 	}
#   ]

# }

# old style


dict_gp_ds, dict_gp_cd, dict_cd_ds = get_sentences_offset_per_cooccurance(final_sentences)

In [ ]:
dict_gp_ds

In [ ]:
def get_cooccurance_evidence(dict_tags, tag_type_1, tag_type_2):
    mined_sentences = []
    for each_sent_map, mappedtags in dict_tags.items():
        for eachtag in mappedtags:
            mini_dict = {}
            eachtag = list(eachtag)
            mini_dict['section'] = section_tags[each_sent_map]

            if tag_type_1 =='GP' or tag_type_2 =='GP':          
                if tag_type_1 in eachtag[0]:
                    mini_dict['t_start'] = eachtag[0][0]
                    mini_dict['t_end']= eachtag[0][1]
                    mini_dict['t']= eachtag[0][3]
                else:
                    mini_dict['t_start'] = eachtag[1][0]
                    mini_dict['t_end']= eachtag[1][1]
                    mini_dict['t']= eachtag[0][3]           

            if tag_type_1 =='DS' or tag_type_2 =='DS':  
                if tag_type_2 in eachtag[0]:
                    mini_dict['d_start'] = eachtag[0][0]
                    mini_dict['d_end']= eachtag[0][1]
                    mini_dict['d']= eachtag[1][3]
                else:
                    mini_dict['d_start'] = eachtag[1][0]
                    mini_dict['d_end']= eachtag[1][1]
                    mini_dict['d']= eachtag[1][3]
                            
            if tag_type_1 =='CD' or tag_type_2 =='CD':  
                if tag_type_1 in eachtag[0]:
                    mini_dict['c_start'] = eachtag[0][0]
                    mini_dict['c_end']= eachtag[0][1]
                    mini_dict['c']= eachtag[1][3]
                else:
                    mini_dict['c_start'] = eachtag[1][0]
                    mini_dict['c_end']= eachtag[1][1]
                    mini_dict['c']= eachtag[1][3]

            mini_dict['text'] = each_sent_map
            mined_sentences.append(mini_dict)
    return mined_sentences


In [ ]:
json_generated = {}
for each_sentence, ml_tags in final_sentences.items():
    try:
        json_generated['pmid'] = 'PMC'+soup.find(attrs={"pub-id-type" : "pmcid"}).text #"article-id"
    except:
        json_generated['pmid'] = 'PMC'

    json_generated['GP_set'] = list(gp_set)
    json_generated['DS_set'] = list(ds_set)
    json_generated['CD_set'] = list(cd_set)
    json_generated['OG_set'] = list(og_set)

# json_generated
json_generated['GP_DS_mined_sentences'] = get_cooccurance_evidence(dict_gp_ds, tag_type_1='GP', tag_type_2='DS')
json_generated['GP_CD_mined_sentences'] = get_cooccurance_evidence(dict_gp_cd, tag_type_1='GP', tag_type_2='CD')
json_generated['CD_DS_mined_sentences'] = get_cooccurance_evidence(dict_cd_ds, tag_type_1='CD', tag_type_2='DS')

In [ ]:
json_generated

In [ ]:
# https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=EXT_ID%3A2109978%20AND%20SRC%3AMED&resultType=core&cursorMark=*&pageSize=25&format=json

import requests
query = '2109978'
url = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=EXT_ID:'+query+' AND SRC:MED&resultType=core&cursorMark=*&pageSize=25&format=json'
response = requests.get(url)
rjson = response.json()

In [ ]:
rjson['resultList']['result'][0]['abstractText']

In [ ]:
xx_

In [ ]:
xx_==yy_

# FP Analysis on test set

In [ ]:
def get_pmc_meta(query):
        
        url = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?'
        params = {'query':query, 'resultType':'core','cursorMark':'*','pageSize':'25','format':'json'}
        response = requests.get(url,params)
        rjson = response.json()

        for rslt in rjson['resultList']['result']:
            pmeta = {}
            pmeta['pmid'] = ''
            pmeta['pmcid'] = ''
            pmeta['mesh'] = []
            pmeta['pmc_title'] = ''
            pmeta['pmc_abstract']= ''
            pmeta['text_urls'] = []
            pmeta['oa'] =  ''
            pmeta['authMan'] = ''
            if 'pmid' in rslt.keys():
                pmeta['pmid'] = rslt['pmid']
                pmids.append(rslt['pmid'])                
            if 'meshHeadingList' in rslt.keys():
                for m in rslt['meshHeadingList']['meshHeading']:
                    if 'meshQualifierList' in m.keys():
                        for q in m['meshQualifierList']['meshQualifier']:
                            pmeta['mesh'].append(m['descriptorName'])
                            pmeta['mesh'].append(q['qualifierName'])
                    else:
                        pmeta['mesh'].append(m['descriptorName'])
            if 'title' in rslt.keys(): pmeta['pmc_title'] = rslt['title']
            if 'abstractText' in rslt.keys(): pmeta['pmc_abstract'] = rslt['abstractText']
            if 'fullTextUrlList' in rslt.keys():            
                for u in rslt['fullTextUrlList']['fullTextUrl']:
                    pmeta['text_urls'].append(u['url'])
            if 'isOpenAccess' in rslt.keys():pmeta['oa'] =  rslt['isOpenAccess']
            if 'authMan' in rslt.keys():pmeta['authMan'] = rslt['authMan']
            if 'pmcid' in rslt.keys() and rslt['pmcid']!='':
                pmeta['pmcid'] = rslt['pmcid']
                pmcids.append(rslt['pmcid'])
                if rslt['isOpenAccess'] == 'Y' or rslt['authMan'] == 'Y':
                #if rslt['isOpenAccess'] == 'Y':    
                    oapids.append(rslt['pmcid'])                        
            pmetas.append(pmeta)
    except:
        print('ERROR IN PMC ID:'+query)
        pass
    return [pmids,pmcids,oapids,pmetas]

In [ ]:
import pandas as pd

path_EPMC_text = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/europePMC-NER/annotations_API/full_sentences/test_annotations/Europe_PMC_annotation.csv'
colNames = ['pmc_id', 'section', 'sentence','ner'] 
    
test_df = pd.read_csv(path_EPMC_text,sep ='\t', names=colNames) 

In [ ]:
from ast import literal_eval

def deleting_epmc_GPS(list_1,del_name):
   
    for sub_list in list_1:
        if del_name in sub_list:
            list_1.remove(sub_list)
    return list_1

In [ ]:
from fuzzywuzzy import fuzz

def remove_FP(epmc_list, ml_json):
    all_ml_gps = []
    if ml_json['annotations']:
        for each_ml_annotation in ml_json['annotations']:
            if each_ml_annotation[2] == 'GP':
                all_ml_gps.append(each_ml_annotation[3])
    
    non_FP_removed =[]
              

    for each_ner in epmc_list:
        if each_ner[2] == 'Gene_Proteins':
            for each_ml_gp in all_ml_gps:         
                score = fuzz.token_set_ratio(each_ml_gp, each_ner[1])
                if score == 100:
                    non_FP_removed.append(each_ner)

    non_gp_tags =  deleting_epmc_GPS(epmc_list,'Gene_Proteins')  

    fp_removed_tags = non_gp_tags+non_FP_removed

    return fp_removed_tags


In [ ]:
def get_spacy_annotations(text_sentence):
    data_dict ={}
    doc = nlp2(text_sentence)
    terms_entities = []
    
    for ent in doc.ents:
        terms_entities.append(
            [ent.start_char, ent.end_char, ent.label_, ent.text])
    
    data_dict['annotations'] = terms_entities
    
    return data_dict

In [ ]:
# from tqdm import tqdm
with open(result_path + 'spacy_fp_removal_80.tsv', 'w', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index,row in tqdm(test_df.iterrows(),total = len(test_df)):
        try:
            ml_annotations = get_spacy_annotations(row['sentence'])
            fp_removed = remove_FP(literal_eval(row['ner']), ml_annotations)
        except ValueError:
            fp_removed =''
            
        public_writer.writerow([row['pmc_id'], row['section'],row['sentence'], fp_removed])   
            
    

In [ ]:
# Convert to IOB format

import sys
import os
import glob

sys.path.append('/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/test Gitlab/epmc-ml-misc-library/')

import capo_tools_lib
import evaluation_epmc_lib




In [ ]:
iob_result_path = result_path+'iob/'
pathlib.Path(iob_result_path).mkdir(parents=True, exist_ok=True)

file_path = result_path + 'spacy_fp_removal_80.tsv'
capo_tools_lib.annotations_api_tagged_sentences_to_IOB(file_path,
                                                       iob_result_path,'spacy_fp_removal_iob.tsv')

In [ ]:
import metrics.ner as ner_metrics


# precision	0.7	0.7	0.72	0.73
# recall	0.53	0.53	0.54	0.55
# f1 score	0.6	0.6	0.62	0.6

#
# print(ner_metrics.semeval_scores_report(gold=epmc_labels, response=ml_labels, digits=2))

root_path = '/mnt/droplet/nfs/gns/literature/machine-learning/'
epmc_path = root_path+'Datasets/NER_Datasets/EBI_standard-IOB/test.csv'
all_tags = ['GP', 'DS', 'OG']

print('################ Annotation Pipeline Results ########################')
CAPO_path = iob_result_path+'spacy_fp_removal_iob.tsv'
for each_tag in all_tags:
    print('############ '+each_tag+' ####################')
    print('\n')
    print(ner_metrics.semeval_report(gold_path=epmc_path, response_path=CAPO_path, targets=[each_tag]))